In [1]:
import pandas
import ast
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from collections import Counter
import json
import numpy as np
import gensim

In [2]:
def train_word2vec():
    data = []
    with open("intermediate.csv", "r", encoding="utf-8") as file:
        file.readline()
        for line in file:
            res = ast.literal_eval(line.split("#")[1][:-1])
            data.append(res)
    model = gensim.models.Word2Vec(
        data,
        size=150,
        window=10,
        min_count=2,
        workers=10)
    model.train(data, total_examples=len(data), epochs=10)
    model.save("word2vec.model")

In [3]:
train_word2vec()

In [4]:
import torch


class Articles(Dataset):
    def __init__(self, filename, classes):
        self.dir = dir
        self.classes = classes
        self.df = pandas.read_csv(filename, delimiter=",", encoding="utf-8")
        self.model = gensim.models.Word2Vec.load("word2vec.model")

    def __getitem__(self, index):
        line = ast.literal_eval(self.df["text"][index])
        text = torch.empty(45,150)
        for i in range(45):
            text[i,:] = torch.from_numpy(self.model.wv[line[i]])
        # on a qu'une seule classe dans le dataset
        # labels = ast.literal_eval(self.df["topic"][index])
        # topic = torch.zeros(self.classes)
        # for label in labels:
        #     topic[label] = 1
        topic = torch.LongTensor([self.df["topic"][index]])
        return text, topic

    def __len__(self):
        return len(self.df)


In [5]:

class TopicClassifier(torch.nn.Module):
    def __init__(self, tokens, classes):
        super(TopicClassifier, self).__init__()
        device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
        self.lstm = torch.nn.LSTM(150, 1024, batch_first=True)
        # replace (1, 1, 512) with (1, 50, 512) for prediction
        self.cell = (torch.zeros(1, 1, 1024).to(device),
                     torch.zeros(1, 1, 1024).to(device))
        self.fc = torch.nn.Linear(1024, classes)

    # dans le dataset un article n'appartient qu'à un seul topic, nous allons modifier le modele pour correspondre à cela
    def forward(self, x):
        l, self.cell = self.lstm(x, self.cell)
        l = l[:, -1, :]
        x = self.fc(l)
        # return torch.sigmoid(x)
        return x
